In [672]:
#libraies we need
import numpy as np 
import pandas as pd 
import plotly.express as px
from sklearn.preprocessing import OneHotEncoder,OrdinalEncoder
#reviewing the data
df=pd.read_csv('train.csv')
l1=df.columns

In [673]:
#manual classiication of columns based wether the data is numerical,nominal or ordinal data 
numerical=df[['LotFrontage','LotArea','MasVnrArea','BsmtFinSF1','BsmtFinSF2','BsmtUnfSF','TotalBsmtSF','1stFlrSF','2ndFlrSF',
              'LowQualFinSF','GrLivArea','BsmtFullBath','BsmtHalfBath','FullBath','HalfBath','BedroomAbvGr','KitchenAbvGr',
              'TotRmsAbvGrd','Fireplaces','GarageCars','GarageArea','WoodDeckSF','OpenPorchSF','EnclosedPorch',
              '3SsnPorch','ScreenPorch','PoolArea','MiscVal','SalePrice']]
nominal=df[['Street','Alley','LandContour','LotConfig','Neighborhood','Condition1',
           'Condition2','BldgType','HouseStyle','RoofStyle','RoofMatl',
           'Exterior1st','Exterior2nd','MasVnrType','Foundation','Heating','CentralAir',
           'GarageType','GarageFinish','MiscFeature','SaleType','SaleCondition']]
ordinal=df[['MSSubClass','MSZoning','LotShape','Utilities','LandSlope','OverallQual','OverallCond'
           ,'YearBuilt','YearRemodAdd','ExterQual','ExterCond','BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinType2',
           'HeatingQC','Electrical','KitchenQual','Functional','FireplaceQu','GarageYrBlt','GarageFinish','GarageQual','GarageCond',
           'PavedDrive','PoolQC','Fence','MoSold','YrSold']]

In [674]:
#Functions to check for dominant columns and excecive nans in order to remove as they are not necessary for the model
def get_low_information_columns(df, dominance_threshold=0.95):
    """
    Finds columns where one category dominates (occurs in >= dominance_threshold proportion of rows).
    Parameters:
    - df: pandas DataFrame
    - column_list: list of columns to check
    - dominance_threshold: float (0 to 1), threshold for dominant category
    
    Returns:
    - List of column names with dominant single-category values
    """
    low_info_cols = []

    for col in df.columns:
        top_freq = df[col].value_counts(normalize=True, dropna=False).values[0]
        if top_freq >= dominance_threshold:
            low_info_cols.append(col)

    return low_info_cols

def get_columns_with_excessive_nans(df, threshold=0.5):
    """
    Returns columns from column_list where the proportion of NaN values exceeds the given threshold.
    Parameters:
    - df: pandas DataFrame
    - column_list: list of column names to check
    - threshold: float between 0 and 1 (e.g. 0.5 means drop if >50% NaNs)
    Returns:
    - List of column names to consider dropping
    """
    total_rows = len(df)
    drop_candidates = []
    for col in df.columns:
        nan_ratio = df[col].isna().sum() / total_rows
        if nan_ratio > threshold:
            drop_candidates.append(col)
    return drop_candidates


In [675]:
#selecting columns to drop based on their dominance and nans number

numerical_drop=['BsmtFinSF1','BsmtFinSF2','BsmtUnfSF','BedroomAbvGr','GarageCars']

ordinal_drops=get_low_information_columns(ordinal,0.7)
nominal_drops=get_low_information_columns(nominal,0.7)

nan_numerical=get_columns_with_excessive_nans(numerical,0.7)
nan_nominal=get_columns_with_excessive_nans(nominal,0.7)
nan_ordinal=get_columns_with_excessive_nans(ordinal,0.7)

tot_num_drop=list(set(numerical_drop+nan_numerical))
tot_nom_drop=list(set(nominal_drops+nan_nominal))
tot_ord_drop=list(set(ordinal_drops+nan_ordinal))
tot_num_drop
#dropat

numerical_v2=numerical.drop(tot_num_drop,axis=1)
nominal_v2=nominal.drop(tot_nom_drop,axis=1)
ordinal_v2=ordinal.drop(tot_ord_drop,axis=1)
ordinal_v2=ordinal_v2.drop('MoSold',axis=1)


In [676]:
#feature engineering
full_bath=['BsmtFullBath','FullBath']
half_bath=['BsmtHalfBath','HalfBath']
tot_area=['LotFrontage','LotArea','MasVnrArea','TotalBsmtSF','1stFlrSF','2ndFlrSF','GarageArea','WoodDeckSF','OpenPorchSF','EnclosedPorch','3SsnPorch','ScreenPorch',
          'PoolArea']
#Adding all th area columns of the house into a single area of the house
numerical_v2['total_area']=numerical_v2[tot_area].sum(axis=1)
numerical_v2.drop(tot_area,axis=1,inplace=True)
#Adding all the full bathrooms and half bathrooms into a single column
numerical_v2['totalbathreoams']=numerical_v2[full_bath].sum(axis=1)+(numerical_v2[half_bath].sum(axis=1)*0.5)
numerical_v2=numerical_v2.drop(full_bath,axis=1)
numerical_v2=numerical_v2.drop(half_bath,axis=1)


significant=['LowQualFinSF','GrLivArea','KitchenAbvGr','TotRmsAbvGrd']#expected signeficant columns from data understanding
numerical_v2

,LowQualFinSF,GrLivArea,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,MiscVal,SalePrice,total_area,totalbathreoams
0,0,1710,1,8,0,0,208500,11886.0,3.5
1,0,1262,1,6,1,0,181500,12962.0,2.5
2,0,1786,1,6,1,0,223500,14836.0,3.5
3,0,1717,1,7,1,0,140000,13032.0,2.0
4,0,2198,1,9,1,0,250000,19149.0,3.5
...,...,...,...,...,...,...,...,...,...
1455,0,1647,1,7,1,0,175000,11079.0,2.5
1456,0,2073,1,7,2,0,210000,17843.0,3.0
1457,0,2340,1,9,2,2500,266500,12912.0,2.0
1458,0,1078,1,5,0,0,142125,12659.0,2.0


In [677]:
# the correlation matrix
corr_matrix = numerical_v2.corr(numeric_only=True)
# Convert it to long format for Plotly
corr_long = corr_matrix.reset_index().melt(id_vars='index')
corr_long.columns = ['Feature1', 'Feature2', 'Correlation']
# Create Plotly heatmap
fig = px.imshow(
    corr_matrix,
    text_auto='.2f',
    color_continuous_scale='RdBu_r',
    title='Correlation Matrix Heatmap',
    aspect='auto'
)

fig.update_layout(
    xaxis_title='Features',
    yaxis_title='Features',
    xaxis_tickangle=-45
)
fig.show()

In [678]:
numerical_v2=numerical_v2.drop('MiscVal',axis=1)
numerical_v2=numerical_v2.drop('GrLivArea',axis=1)
numerical_v2

,LowQualFinSF,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,SalePrice,total_area,totalbathreoams
0,0,1,8,0,208500,11886.0,3.5
1,0,1,6,1,181500,12962.0,2.5
2,0,1,6,1,223500,14836.0,3.5
3,0,1,7,1,140000,13032.0,2.0
4,0,1,9,1,250000,19149.0,3.5
...,...,...,...,...,...,...,...
1455,0,1,7,1,175000,11079.0,2.5
1456,0,1,7,2,210000,17843.0,3.0
1457,0,1,9,2,266500,12912.0,2.0
1458,0,1,5,0,142125,12659.0,2.0


In [679]:
#nominal encoding 
encoded_nominal=pd.DataFrame()
for col in nominal_v2.columns:
    ohe = OneHotEncoder(sparse_output=False, drop='first')  # drop first to avoid dummy trap
    encoded_array = ohe.fit_transform(nominal_v2[[col]])     # double brackets to keep it as DataFrame
    encoded_cols = ohe.get_feature_names_out([col])  # get new column names
    encoded_part = pd.DataFrame(encoded_array, columns=encoded_cols)
    encoded_nominal = pd.concat([encoded_nominal, encoded_part], axis=1)
encoded_nominal

,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_CollgCr,Neighborhood_Crawfor,Neighborhood_Edwards,Neighborhood_Gilbert,Neighborhood_IDOTRR,Neighborhood_MeadowV,...,Foundation_Wood,GarageType_Attchd,GarageType_Basment,GarageType_BuiltIn,GarageType_CarPort,GarageType_Detchd,GarageType_nan,GarageFinish_RFn,GarageFinish_Unf,GarageFinish_nan
0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1456,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1457,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1458,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [680]:
#ordinal encoding
#small function to show all the unique values of each column (numerical are sorted) in order to make the order manually for the encoder 
'''for col in ordinal_v2.columns:
    uniques = sorted(ordinal_v2[col].dropna().unique())

    cleaned_uniques = []
    for val in uniques:
        if isinstance(val, (np.integer, int)):
            cleaned_uniques.append(int(val))
        elif isinstance(val, (np.floating, float)):
            # Convert float to int if it's a whole number like 1900.0
            cleaned_uniques.append(int(val) if val.is_integer() else float(val))
        else:
            cleaned_uniques.append(val)

    print(f"{col}: {cleaned_uniques}")
'''


orders_list=[[20, 30, 40, 45, 50, 60, 70, 75, 80, 85, 90, 120, 160, 180, 190],
             ['Reg','IR1', 'IR2', 'IR3'],
             [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
             [1, 2, 3, 4, 5, 6, 7, 8, 9],
             [1872, 1875, 1880, 1882, 1885, 1890, 1892, 1893, 1898, 1900, 1904, 1905, 1906, 1908,
              1910, 1911, 1912, 1913, 1914, 1915, 1916, 1917,1918, 1919, 1920, 1921, 1922, 1923, 
              1924, 1925, 1926, 1927, 1928, 1929, 1930, 1931, 1932, 1934, 1935, 1936, 1937, 1938,
              1939, 1940,1941, 1942, 1945, 1946, 1947, 1948, 1949, 1950, 1951, 1952, 1953, 1954, 
              1955, 1956, 1957, 1958, 1959, 1960, 1961, 1962, 1963, 1964,1965, 1966, 1967, 1968, 
              1969, 1970, 1971, 1972, 1973, 1974, 1975, 1976, 1977, 1978, 1979, 1980, 1981, 1982,
              1983, 1984, 1985,1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 
              1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010],
             [1950, 1951, 1952, 1953, 1954, 1955, 1956, 1957, 1958, 1959, 1960, 1961, 1962, 1963, 
              1964, 1965, 1966, 1967, 1968, 1969, 1970,1971, 1972, 1973, 1974, 1975, 1976, 1977, 
              1978, 1979, 1980, 1981, 1982, 1983,1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 
              1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005,
              2006, 2007, 2008, 2009, 2010],
             ['Ex', 'Gd', 'TA', 'Fa'],
             ['Ex', 'Gd', 'TA', 'Fa'],
             [ 'Gd','Av', 'Mn', 'No'],
             ['GLQ','ALQ', 'BLQ', 'Rec', 'LwQ',  'Unf'],
             ['Ex', 'Gd', 'TA', 'Fa', 'Po'],
             ['Ex', 'Gd', 'TA', 'Fa'],
             ['Ex', 'Gd', 'TA', 'Fa', 'Po'],
             [1900, 1906, 1908, 1910, 1914, 1915, 1916, 1918, 1920, 1921, 1922, 1923, 1924, 1925,
              1926, 1927, 1928, 1929, 1930, 1931, 1932, 1933, 1934, 1935, 1936, 1937, 1938, 1939,
              1940, 1941, 1942, 1945, 1946, 1947, 1948, 1949, 1950, 1951, 1952, 1953, 1954, 1955,
              1956, 1957, 1958, 1959, 1960, 1961, 1962, 1963, 1964, 1965, 1966, 1967, 1968, 1969,
              1970, 1971, 1972, 1973, 1974, 1975, 1976, 1977, 1978, 1979, 1980, 1981, 1982, 1983,
              1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997,
              1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010],
              ['Fin', 'RFn', 'Unf'],
               [2006, 2007, 2008, 2009, 2010]]

ordinal_cols=ordinal_v2.columns
encoder = OrdinalEncoder(categories=orders_list,handle_unknown='use_encoded_value', unknown_value=-1)
ordinal_v2[ordinal_cols] = encoder.fit_transform(ordinal_v2[ordinal_cols])
ordinal_encoded=ordinal_v2
ordinal_encoded



,MSSubClass,LotShape,OverallQual,OverallCond,YearBuilt,YearRemodAdd,ExterQual,BsmtQual,BsmtExposure,BsmtFinType1,HeatingQC,KitchenQual,FireplaceQu,GarageYrBlt,GarageFinish,YrSold
0,5.0,0.0,6.0,4.0,104.0,53.0,1.0,1.0,3.0,0.0,0.0,1.0,-1.0,89.0,1.0,2.0
1,0.0,0.0,5.0,7.0,77.0,26.0,2.0,1.0,0.0,1.0,0.0,2.0,2.0,62.0,1.0,1.0
2,5.0,1.0,6.0,4.0,102.0,52.0,1.0,1.0,2.0,0.0,0.0,1.0,2.0,87.0,1.0,2.0
3,6.0,1.0,6.0,4.0,19.0,20.0,2.0,2.0,3.0,1.0,1.0,1.0,1.0,84.0,2.0,0.0
4,5.0,1.0,7.0,4.0,101.0,50.0,1.0,1.0,1.0,0.0,0.0,1.0,2.0,86.0,1.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,5.0,0.0,5.0,4.0,100.0,50.0,2.0,1.0,3.0,5.0,0.0,2.0,2.0,85.0,1.0,1.0
1456,0.0,0.0,5.0,5.0,79.0,38.0,2.0,1.0,3.0,1.0,2.0,2.0,2.0,64.0,2.0,4.0
1457,6.0,0.0,6.0,8.0,44.0,56.0,0.0,2.0,3.0,0.0,0.0,1.0,1.0,29.0,1.0,4.0
1458,0.0,0.0,4.0,5.0,51.0,46.0,2.0,2.0,2.0,0.0,1.0,1.0,-1.0,36.0,2.0,4.0


In [681]:
#final data frame (needs scaling and ready for ML)
final=pd.concat([numerical_v2,encoded_nominal,ordinal_encoded],axis=1)
final

,LowQualFinSF,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,SalePrice,total_area,totalbathreoams,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,...,ExterQual,BsmtQual,BsmtExposure,BsmtFinType1,HeatingQC,KitchenQual,FireplaceQu,GarageYrBlt,GarageFinish,YrSold
0,0,1,8,0,208500,11886.0,3.5,0.0,0.0,0.0,...,1.0,1.0,3.0,0.0,0.0,1.0,-1.0,89.0,1.0,2.0
1,0,1,6,1,181500,12962.0,2.5,0.0,0.0,0.0,...,2.0,1.0,0.0,1.0,0.0,2.0,2.0,62.0,1.0,1.0
2,0,1,6,1,223500,14836.0,3.5,0.0,0.0,0.0,...,1.0,1.0,2.0,0.0,0.0,1.0,2.0,87.0,1.0,2.0
3,0,1,7,1,140000,13032.0,2.0,0.0,0.0,0.0,...,2.0,2.0,3.0,1.0,1.0,1.0,1.0,84.0,2.0,0.0
4,0,1,9,1,250000,19149.0,3.5,0.0,0.0,0.0,...,1.0,1.0,1.0,0.0,0.0,1.0,2.0,86.0,1.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,0,1,7,1,175000,11079.0,2.5,0.0,0.0,0.0,...,2.0,1.0,3.0,5.0,0.0,2.0,2.0,85.0,1.0,1.0
1456,0,1,7,2,210000,17843.0,3.0,0.0,0.0,0.0,...,2.0,1.0,3.0,1.0,2.0,2.0,2.0,64.0,2.0,4.0
1457,0,1,9,2,266500,12912.0,2.0,0.0,0.0,0.0,...,0.0,2.0,3.0,0.0,0.0,1.0,1.0,29.0,1.0,4.0
1458,0,1,5,0,142125,12659.0,2.0,0.0,0.0,0.0,...,2.0,2.0,2.0,0.0,1.0,1.0,-1.0,36.0,2.0,4.0
